In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder 
from sklearn.decomposition import PCA
from sklearn import preprocessing 
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn import cluster
pd.set_option('display.max_rows', None) # 省略画图中的明细过程
plt.rcParams['font.sans-serif'] = ['Sim Hei']#中文编码

## K—means聚类渠道广告效果分析

### 一、数据探索

本项目，通过各类广告渠道90天内额日均UV，平均注册率、平均搜索率、访问深度、平均停留时长、订单转化率、投放时间、素材类型、广告类型、合作方式、广告尺寸和广告卖点等特征，将渠道分类，找出每类渠道的重点特征，为业务讨论和数据分析提供支持。
数据13个维度介绍

渠道代号：渠道唯一标识

日均UV：每天的独立访问量

平均注册率=日均注册用户数/平均每日访问量

平均搜索量：每个访问的搜索量

访问深度：总页面浏览量/平均每天的访问量

平均停留时长=总停留时长/平均每天的访问量

订单转化率=总订单数量/平均每天的访客量

投放时间：每个广告在外投放的天数

素材类型：‘jpg’ ‘swf’ ‘gif’ ‘sp’

广告类型：banner. tips. 不确定. 横幅. 暂停

合作方式：‘roi’ ‘cpc’ ‘cpm’ ‘cpd’

广告尺寸：‘14040’ ‘308388’ ‘450300’ ‘60090’ ‘480360’ ‘960126’ ‘900120’
‘390270’

广告卖点：打折. 满减. 满赠. 秒杀. 直降. 满返

In [ ]:
raw_data = pd.read_csv('ad_performance.csv')
raw_data.head()

In [ ]:
raw_data.info()

In [ ]:
# 可以用pandas_profiling,更全面
from pandas_profiling import ProfileReport
ProfileReport(raw_data)

## 数据清洗

In [ ]:
# 观察Unnamed列
len(raw_data['Unnamed: 0'].unique()) == len(raw_data['Unnamed: 0'])
# drop掉'Unnamed: 0'
raw_data = raw_data.drop(['Unnamed: 0'],axis=1)
raw_data

In [ ]:
# 查看缺失情况
raw_data.isnull().sum()/len(raw_data['平均停留时间'])
# 查看具体缺失行
raw_data[raw_data['平均停留时间'].isnull()]
# 处理缺失值
raw_data = raw_data.dropna()

In [ ]:
# 比率指标容易产生辛普森悖论，还原
raw_data['总UV']=raw_data['日均UV']*raw_data['投放总时间']
raw_data['注册总量']=raw_data['日均UV']*raw_data['平均注册率']*raw_data['投放总时间']
raw_data['总搜索量']=raw_data['平均搜索量']*raw_data['总UV']
raw_data['总页面浏览量']=raw_data['总UV']*raw_data['访问深度']
raw_data['总停留时间']=raw_data['总UV']*raw_data['平均停留时间']
raw_data['总订单数量']=raw_data['总UV']*raw_data['订单转化率']
raw_data['投放天数']=raw_data['投放总时间']
raw_data.head()

In [ ]:
# drop掉原来的列
drop_list = ['日均UV','平均注册率','平均搜索量','访问深度','平均停留时间','订单转化率','投放总时间']
df = raw_data.drop(drop_list,axis=1)
df

In [ ]:
# 观察指标间的关系，其实在ProfileReport里有展示，但热力图更直观
import seaborn as sns
# pd.set_option('display.max_rows', None) # 省略画图中的明细过程
fig, ax = plt.subplots(figsize=(10,10),dpi=100)
plt.rcParams['font.sans-serif'] = ['Sim Hei']#中文编码
corr = df.corr().round(2)
sns.set_style("darkgrid",{"font.sans-serif":['KaiTi', 'Arial']})
sns.heatmap(corr,cmap='Blues',annot=True)

In [ ]:
# 可以看到数值变量之间相关性较强，可能产生较强的共线性，应用PCA降维
#分别处理label类和value类
label_list = ['素材类型','广告类型','合作方式','广告尺寸','广告卖点'] 
label_dummies = pd.get_dummies(df[label_list])

In [ ]:
label_model = OneHotEncoder(sparse=False)
label_matrix = label_model.fit_transform(df[label_list])
label_matrix

In [ ]:
value_list = df.drop(label_list,axis = 1)
value_list = value_list.drop('渠道代号',axis=1)
value_list = value_list.columns

In [ ]:
value_list

In [ ]:
# 标准化
from sklearn.preprocessing import Normalizer
model = Normalizer()
value_scaled = model.fit_transform(df[value_list])
value_scaled

In [ ]:
# PCA降维
X_matrix = np.hstack((value_scaled,label_matrix))
pca_model = PCA(n_components=0.80) #要求降维后的数据保留90%的信息
after_pca = pd.DataFrame(pca_model.fit_transform(X_matrix))
after_pca

In [ ]:
col_list = ['col'+str(i) for i in range(1,len(after_pca.columns)+1)]
after_pca.columns = col_list
after_pca

## K-maens聚类

In [ ]:
# 手肘法确定K值
Elbow = KElbowVisualizer(KMeans(),k=(4,12))
Elbow.fit(after_pca)
Elbow.show()

In [ ]:
# 聚类，k=6
k_means_result = KMeans(7).fit_predict(after_pca)
k_re = pd.DataFrame(k_means_result,columns=['clusters'])
new_df = pd.concat((df,k_re),axis=1)
new_df

## 分析结果

In [ ]:
sns.countplot(new_df['clusters']).set_title("Distribution Of The Clusters")

### 成本分析

In [ ]:
df_cost = new_df.groupby(['clusters'],as_index=False)['投放天数'].sum()
sns.barplot(x='clusters',y='投放天数',data=df_cost)
for x,y in zip(df_cost['clusters'],df_cost['投放天数']):
    plt.text(x=x,y=y,s='{:.0f}'.format(y),ha='center')

可以看到，投放天数主要集中于1，2，3渠道

In [ ]:
# 交叉合作方式
# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']#中文编码
sns.boxplot(x='clusters',y='投放天数',hue='合作方式',data=new_df)

### 效果分析

In [ ]:
df_register = new_df.groupby(['clusters'],as_index=False)['注册总量'].sum()
sns.barplot(x='clusters',y='注册总量',data=df_register)
# # 增加数值标签
for x,y in zip(df_register['clusters'],df_register['注册总量']):
    plt.text(x=x,y=y,s='{:.0f}'.format(y),ha='center')

In [ ]:
# 订单数,转化率
df_order = new_df.groupby(['clusters'],as_index=False)['总订单数量'].sum()
sns.barplot(x='clusters',y='总订单数量',data=df_order)
# # 增加数值标签
for x,y in zip(df_order['clusters'],df_order['总订单数量']):
    plt.text(x=x,y=y,s='{:.0f}'.format(y),ha='center')

In [ ]:
# 综合上述指标
df_all = pd.concat([df_cost.set_index('clusters'),df_register.set_index('clusters'),df_order.set_index('clusters')],axis=1)
df_all['单位投放注册率'] = df_all['注册总量']/df_all['投放天数']
df_all['单位投放订单数'] = df_all['总订单数量']/df_all['投放天数']
df_all['转化率'] = df_all['总订单数量']/df_all['注册总量']
df_all

In [ ]:
sns.barplot(x=df_all.index,y=df_all['单位投放注册率'])
for x,y in zip(df_all.index,df_all['单位投放注册率']):
    plt.text(x=x,y=y,s='{:.4f}'.format(y),ha='center')

In [ ]:
sns.barplot(x=df_all.index,y=df_all['单位投放订单数'])
for x,y in zip(df_all.index,df_all['单位投放订单数']):
    plt.text(x=x,y=y,s='{:.4f}'.format(y),ha='center')

In [ ]:
sns.barplot(x=df_all.index,y=df_all['转化率'])
for x,y in zip(df_all.index,df_all['转化率']):
    plt.text(x=x,y=y,s='{:.4f}'.format(y),ha='center')

In [ ]:
## 通过箱线图看看离群点
f = plt.figure(figsize=(16,5))
f.add_subplot(1,2,1)
sns.boxplot(x='clusters',y='注册总量',data=new_df).set_title('分渠道注册量')
f.add_subplot(1,2,2)
sns.boxplot(x='clusters',y='总订单数量',data=new_df).set_title('分渠道订单量')
plt.show()

In [ ]:
#剩余指标
f = plt.figure(figsize=(16,5))
f.add_subplot(1,3,1)
sns.boxplot(x='clusters',y='总搜索量',data=new_df).set_title('分渠道搜索量')
f.add_subplot(1,3,2)
sns.boxplot(x='clusters',y='总页面浏览量',data=new_df).set_title('分渠道页面浏览量')
f.add_subplot(1,3,3)
sns.boxplot(x='clusters',y='总停留时间',data=new_df).set_title('分渠道停留时间')
plt.show()

In [ ]:
sns.catplot(x="广告类型", col="clusters", col_wrap=4,data=new_df,kind="count", height=5, aspect=.8)
sns.catplot(x="广告卖点", col="clusters", col_wrap=4,data=new_df,kind="count", height=5, aspect=.8)
plt.show()

In [ ]:
结论：
聚类1（标签0）：欠佳
投入低，产出低
建议：降低投放比例

聚类2（标签1）：欠佳
投入高，属于主体渠道，但产出低
建议：低性价比，降低投放比例

聚类3（标签2）：优质渠道
引流注册效果好，转化效果好，存在搜索离群值，该类别中存在及其优质个体
建议：增加投放比例，可考虑作为主要投放渠道

聚类4（标签3）：欠佳
投入高，引流效果和转化效果都不太行
建议：性价比低，降低投放比例

聚类5（标签3）：引流好
搜索量，浏览时间，停留时间整体最佳，有较好的注册率与转化率，渠道网络
建议：增加投放比例，进一步挖掘价值

聚类6（标签3）：优质渠道
投入不高，引流注册最高，转化效果最高，搜索量，浏览时间，停留时间处于平均水平，存在搜索离群值，该类别中存在及其优质个体，渠道网络
建议：增加投放比例，进一步挖掘价值

聚类7（标签3）：欠佳
投入低，产出最低
建议：降低投放比例